![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and Google `flan-t5-xxl` to analyze sentiments of legal documents

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate support of sentiment analysis in watsonx. It introduces commands for data retrieval and model testing.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to use `google/flan-t5-xxl` model to analyze sentiments of legal documents.

## Use case & dataset

One of the key use cases of legal sentiment analysis is in assisting legal professionals in predicting case outcomes. By analyzing the sentiment expressed in previous court decisions and related documents, sentiment analysis algorithms can identify patterns and correlations between the sentiment and the final verdict. This can help lawyers and judges in assessing the strength of legal arguments, evaluating the potential impact of public opinion on the case, and making more accurate predictions about the likely outcome of ongoing cases.
The dataset consists of two colums; the phrases and the sentiments.

## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Data loading](#data)
- [Foundation Models on watsonx](#models)
- [Model testing](#predict)
- [Score](#score)
- [Summary](#summary)

<a id="setup"></a>
##  Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


### Install and import the `datasets` and dependecies

In [1]:
!pip install wget | tail -n 1
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [2]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ··········


### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Please enter your project_id (hit enter): e3ba4cfd-25ab-4774-b6fe-63001a3ed8f7


<a id="data"></a>
## Data loading

Download the `legal documents` dataset.

In [4]:
import wget

filename = 'Legal_Sentences.csv'
url = 'https://raw.githubusercontent.com/kmokht1/Datasets/main/Legal_Sentences.csv'

if not os.path.isfile(filename):
    wget.download(url, out=filename)

Read the data.

In [5]:
import pandas as pd

data = pd.read_csv("Legal_Sentences.csv", index_col=0 )
data = data[['Phrase','Sentiment']]
data.head()

,Phrase,Sentiment
0,Getting nowhere with surplusage,-1
1,But the Court nowhere suggested that it would ...,-1
2,Petitioners objection to shaving his beard cla...,-1
3,That result clashes with everything else,-1
4,the tolerable duration of police inquiries in ...,0


Prepare dataset label map.

In [6]:
label_map = {
    -1: "negative",
    0: "neutral",
    1: "positive"
}

Inspect data sample.

In [7]:
data.value_counts(['Sentiment'])

Sentiment
-1           282
 1           172
 0           122
dtype: int64

Split the data into training and test sets.

In [8]:
from sklearn.model_selection import train_test_split

data_train, data_test, y_train, y_test = train_test_split(data['Phrase'],
                                                    data['Sentiment'],
                                                    test_size=0.3,
                                                    random_state=33,
                                                    stratify=data['Sentiment'])
data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under ModelTypes class.
For more information refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

In [9]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT']


You need to specify `model_id` that will be used for inferencing:

In [10]:
model_id = ModelTypes.FLAN_T5_XXL

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#metanames.GenTextParamsMetaNames).

In [11]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY: 1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [12]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id)

### Model's details

In [13]:
model.get_details()

{'model_id': 'google/flan-t5-xxl',
 'label': 'flan-t5-xxl-11b',
 'provider': 'Google',
 'source': 'Hugging Face',
 'short_description': 'flan-t5-xxl is an 11 billion parameter model based on the Flan-T5 family.',
 'long_description': 'flan-t5-xxl (11B) is an 11 billion parameter model based on the Flan-T5 family. It is a pretrained T5 - an encoder-decoder model pre-trained on a mixture of supervised / unsupervised tasks converted into a text-to-text format, and fine-tuned on the Fine-tuned Language Net (FLAN) with instructions for better zero-shot and few-shot performance.',
 'tier': 'class_2',
 'number_params': '11b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'retrieval_augmented_generation',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'quality': 4}},
  {'id': 'summarization', 'ratings': {'quality': 4}},
  {'id': 'retrieval_augmented_generation', 'ratings': {'quality': 3}},
  {'id': 'class

<a id="predict"></a>
##  Find legal documents sentiments

Define instructions for the model.

In [14]:
instruction="""Determine the sentiment of the sentense.
Use either 'positive', 'negative','neutral'.Use the provided examples as a template. """

Prepare model inputs for zero-shot example - use below zero_shot_inputs.

In [15]:
zero_shot_inputs = [{"input": text} for text in data_test['Phrase']]
for i in range(2):
    print(f"The sentence example {i+1} is:\n\t {zero_shot_inputs[i]['input']}\n")

The sentence example 1 is:
	 The Court rejects the CCAs conclusion that Moore failed to make the requisite showings with respect to intellectual functioning

The sentence example 2 is:
	 He argues on appeal that had Defendants written truthful reports, or testified truthfully in deposition



Prepare model inputs for few-shot examples - use below few_shot_inputs.

In [16]:
data_train_and_labels=data_train.copy()
data_train_and_labels['Sentiment']=y_train

In [17]:
few_shot_example=[]
few_shot_examples=[]
for phrase,sentiment in data_train_and_labels.groupby('Sentiment').apply(lambda x: x.sample(2)).values:
    few_shot_example.append(f"\tsentence:\t{phrase}\n\tsentiment: {sentiment}\n")
few_shot_examples=[''.join(few_shot_example)]

In [18]:
few_shot_inputs_ = [{"input": text} for text in data_test['Phrase'].values]
for i in range(2):
    print(f"The sentence example {i+1} is:\n {few_shot_inputs_[i]['input']}\n")
    print(f"\tSentiment: {y_test[i]}\n")

The sentence example 1 is:
 The Court rejects the CCAs conclusion that Moore failed to make the requisite showings with respect to intellectual functioning

	Sentiment: -1

The sentence example 2 is:
 He argues on appeal that had Defendants written truthful reports, or testified truthfully in deposition

	Sentiment: -1



### Generate the sentiments of legal documents using `google/flan-t5-xxl` model.


Get the docs summaries.

In [19]:
results = []
for inp in few_shot_inputs_[:2]:
    results.append(model.generate(" ".join([instruction+few_shot_examples[0], inp['input']]))["results"][0])

Explore model output.

In [20]:
import json

print(json.dumps(results, indent=2))

[
  {
    "generated_text": "neutral",
    "generated_token_count": 1,
    "input_token_count": 241,
    "stop_reason": "max_tokens"
  },
  {
    "generated_text": "neutral",
    "generated_token_count": 1,
    "input_token_count": 241,
    "stop_reason": "max_tokens"
  }
]


<a id="score"></a>
## Score the model

**Note:** To run the Score section for model scoring on the whole financial phrasebank dataset, please transform following `markdown` cells to `code` cells.
Have in mind that scoring model on the whole test set can take significant amount of time.

Get the true labels.

In [21]:
y_true = [label_map[label] for label in y_test.values[:2]]
y_true

['negative', 'neutral']

Get the prediction labels.

In [22]:
y_true = [label_map[label] for label in y_test.values[:2]]
y_true

['negative', 'neutral']

In [24]:
y_pred = [result['generated_text'] for result in results]
y_pred

['neutral', 'neutral']

Calculate the accuracy score.

In [25]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_pred, y_true))

0.5


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.

 You learned how to find sentiments of legal documents with Google's `flan-t5-xxl` on watsonx.

Check out our _[Online Documentation](https://ibm.github.io/watson-machine-learning-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts.


### Authors:
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.